In [11]:
from clickhouse_driver import Client
import io
import pandas as pd
import numpy as np
from pymongo import MongoClient
from bson.objectid import ObjectId
import datetime
import psycopg2
import requests
import json
import time
from requests_futures.sessions import FuturesSession
from backend_logic import *
session = FuturesSession()

offerid = '5d5d4e2c7e76801af8637444'
maxRivalsCount = 12
sqiDiffCoef = 100
maxPos = 10
minCountInSerm = 6

offer = getOffer(offerid)

# Получить ключи из КП
keywords = []
keywordTexts = []
for keyword in offer['KeywordsToSave']:
    text = keyword[1]['Text'].lower()
    if "'" in text:
        text = text.replace("'", "''")
    keywords.append({
        'id': keyword[0],
        'text': text
    })
    keywordTexts.append(text)
keywordsStr = u"'" + u"', '".join(keywordTexts) + u"'"

# Получить дату создания из КП
startDate = offer['CreationTime'] - timedelta(days=5)
startDate = startDate.strftime('%Y-%m-%d')

# Получить домен из КП
offerDomain = offer['Domain'].replace('www.', '')

# Получить регионы из КП
offerRegions = getOfferRegions(offer)

# Получить id регионов из КП для Яндекса и Гугла
regions = getRegionIds(offerRegions)
regionsStr = u"'" + u"', '".join([x['yaId'] for x in regions])\
        + u"', '" + u"', '".join([x['gId'] for x in regions]) + u"'"

# Получить конкурентов из базы
query =f"""
    select domain, count(*)
    from secache3
    where
        keyword in ({keywordsStr})
        and date >= '{startDate}'
        and regionId in ({regionsStr})
        and position <= {maxPos}
        and isMobile = 0
    group by domain
    having count(*) >= {minCountInSerm}
    order by count(*) desc
    limit {maxRivalsCount}
"""

client = Client(host = '192.168.10.101', user = 'read_only', password = 'User_ro', settings={'max_query_size':500000})
data = client.execute(query)
client.disconnect()

# Получить популярные домены из файла конфига
excluded_domains = []
with io.open("./popular-domains.txt", mode="r", encoding="utf-8") as f:
    for line in f:
        excluded_domains.append(str.strip(line))
excluded_domains.append(offerDomain)

# Заполнить данные по доменам
domains = []
ourSqi = getSqi(offerDomain)
hasOfferDomain = False
for domain in data:
    domainSqi = getSqi(domain[0])
    isExcluded = False
    for ex in excluded_domains:
        if ex == domain[0] or '.' + ex in domain[0]:
            isExcluded = True
    domains.append({
        'domain': domain[0],
        'countInSerm': domain[1],
        'isExcluded': isExcluded,
        'sqi': getSqiPromise(domain[0]),
        'isOfferDomain': domain[0] == offerDomain
    })
    if domain[0] == offerDomain:
        hasOfferDomain = True
if not hasOfferDomain:
    domains.append({
        'domain': offerDomain,
        'countInSerm': 0,
        'isExcluded': False,
        'sqiDiff': 1,
        'sqi': ourSqi,
        'isOfferDomain': True
    })
for domain in domains:
    try:
        sqi = getSqiResult(domain['sqi'], domain['domain'])
        domain['sqi'] = sqi,
        domain['sqiDiff'] = sqi / ourSqi
        if(domain['sqiDiff']) > sqiDiffCoef:
            domain['isExcluded'] = True
    except:
        domain['sqi'] = 0
        domain['sqiDiff'] = 0

In [12]:
domains

[{'domain': 'e-kc.ru',
  'countInSerm': 1684,
  'isExcluded': False,
  'sqi': (750,),
  'isOfferDomain': False,
  'sqiDiff': 3.125},
 {'domain': 'elcable.ru',
  'countInSerm': 950,
  'isExcluded': False,
  'sqi': (450,),
  'isOfferDomain': False,
  'sqiDiff': 1.875},
 {'domain': 'cable.ru',
  'countInSerm': 906,
  'isExcluded': False,
  'sqi': (1000,),
  'isOfferDomain': False,
  'sqiDiff': 4.166666666666667},
 {'domain': 'etm.ru',
  'countInSerm': 886,
  'isExcluded': True,
  'sqi': (7300,),
  'isOfferDomain': False,
  'sqiDiff': 30.416666666666668},
 {'domain': 'k-ps.ru',
  'countInSerm': 715,
  'isExcluded': False,
  'sqi': (400,),
  'isOfferDomain': False,
  'sqiDiff': 1.6666666666666667},
 {'domain': 'tinko.ru',
  'countInSerm': 589,
  'isExcluded': False,
  'sqi': (3400,),
  'isOfferDomain': False,
  'sqiDiff': 14.166666666666666},
 {'domain': 'podolskkabel.ru',
  'countInSerm': 569,
  'isExcluded': False,
  'sqi': (350,),
  'isOfferDomain': False,
  'sqiDiff': 1.4583333333333333

In [10]:
query = """select domain
    from secache3
    where
        keyword in ('КГВВнг')
        and date >= '2019-08-16'
    limit 12"""
client = Client(host = '192.168.10.101', user = 'read_only', password = 'User_ro', settings={'max_query_size':500000})
data = client.execute(query)
client.disconnect()
data

[]

In [ ]:
from backend_logic import *
offerid = '5d56807f7e76808f64a3b555'
for key in getFilterRivalsObjectByOfferId(offerid):
    print(key)

In [ ]:
all_keywords, notRivals = getKeywordsRivals(keywords, rivals, maxPos, startDate, keywordsStr, regionsStr, regions, minCountInSerm)
all_keywords

In [ ]:
minKeywordRivals = 2
keywordsToDelete = getKeywordsToRemove(all_keywords, minKeywordRivals, regions)
keywordsToDelete

In [ ]:
filename = 'Rivals ' + offerid + ' ' + 'manual' + '.xlsx'
info = [{
    'Id КП': offerid,
    'Макс. кол-во конкурентов': maxRivalsCount,
    'Макс. отклонение ИКС': sqiDiffCoef,
    'Макс. позиция для поиска конкурентов': maxPos,
    'Мин. кол-во раз, сколько сайт должен встречаться в выдаче, чтобы считаться конкурентом': minCountInSerm,
    'Мин. кол-во конкурентов для запроса': minKeywordRivals
}]
keywordsToDeleteForExcel = []
for item in keywordsToDelete:
    for key in item['keywordsToDelete']:
        keywordsToDeleteForExcel.append({
            'Регион': item['region'],
            'Ключ для удаления': key
        })
writer = pd.ExcelWriter('./downloadable_files/' + filename)
info_df = pd.DataFrame(info)
info_df.to_excel(writer, index=False, sheet_name='Метаданные', freeze_panes = (1,0))
regions_df = pd.DataFrame(regions).rename(columns={'name': 'Регион'})['Регион'].to_frame()
regions_df.to_excel(writer, index=False, sheet_name='Регионы', freeze_panes = (1,0))
rivals_df = pd.DataFrame(rivals).rename(columns={
    'domain': 'Сайт',
    'isExcluded': 'Не конкурент',
    'countInSerm': 'Встречается в выдаче',
})[['Сайт', 'Не конкурент', 'Встречается в выдаче']]
rivals_df.to_excel(writer, index=False, sheet_name='Конкуренты', freeze_panes = (1,0))
all_keywords_df = pd.DataFrame(keywords).rename(columns={
    'keyword': 'Запрос',
    'regionName': 'Регион',
    'searchEngine': 'ПС',
    'isMobile': 'Мобильная выдача',
    'rivalsCount': 'Кол-во конкурентов'
})[['Запрос', 'Кол-во конкурентов', 'Регион', 'ПС', 'Мобильная выдача']]
all_keywords_df.to_excel(writer, index=False, sheet_name='Запросы', freeze_panes = (1,0))
keywordsToDeleteForExcel_df = pd.DataFrame(keywordsToDeleteForExcel)
keywordsToDeleteForExcel_df.to_excel(writer, index=False, sheet_name='Запросы для удаления', freeze_panes = (1,0))
writer.save()
print(filename)

In [ ]:
keywordsToDeleteForExcel

In [ ]:
keywordsToDeleteForExcel_df = pd.DataFrame(keywordsToDeleteForExcel)
all_keywords_df = pd.DataFrame(keywords).rename(columns={
    'keyword': 'Запрос',
    'regionName': 'Регион',
    'searchEngine': 'ПС',
    'isMobile': 'Мобильная выдача',
    'rivalsCount': 'Кол-во конкурентов'
})[['Запрос', 'Кол-во конкурентов', 'Регион', 'ПС', 'Мобильная выдача']]
keywordsToDeleteForExcel_df

In [ ]:
import codecs
import json
from backend_logic import *
getOffer('5d5d6def7e76801af8637446')